<a href="https://colab.research.google.com/github/anilcemalyasar/NLP-CaseStudy/blob/main/nlp_casestudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
import json
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
# Drive'a bağlanmak için bu satırı çalıştırmak gerekiyor
from google.colab import drive
drive.mount('/content/drive')

In [5]:
PATH = "/content/drive/MyDrive/NLP-PatikaCaseStudy/data.json"

# JSON dosyasından kitap bilgilerini okuyoruz
try:
    with open(PATH, "r", encoding="utf-8") as f:
        data = json.load(f)
    print("JSON verisi başarıyla okundu")
except Exception as e:
    print(f"Hata: {e}")

JSON verisi başarıyla okundu


In [6]:
# Dataframe formatına getiriyoruz
df = pd.DataFrame(data)

In [7]:
# Verilere genel bir bakış kazanmak adına baştan 10 veriyi bastırıyoruz
df.head(10)

,name,price,starRating,description,stockStatus,stockAmount
0,A Light in the Attic,£51.77,Three,It's hard to imagine a world without A Light i...,False,22
1,Tipping the Velvet,£53.74,One,"""Erotic and absorbing...Written with starling ...",False,20
2,Soumission,£50.10,One,"Dans une France assez proche de la nôtre, un h...",False,20
3,Sharp Objects,£47.82,Four,"WICKED above her hipbone, GIRL across her hear...",False,20
4,Sapiens: A Brief History of Humankind,£54.23,Five,From a renowned historian comes a groundbreaki...,False,20
5,The Requiem Red,£22.65,One,Patient Twenty-nine.A monster roams the halls ...,False,19
6,The Dirty Little Secrets of Getting Your Dream...,£33.34,Four,Drawing on his extensive experience evaluating...,False,19
7,The Coming Woman: A Novel Based on the Life of...,£17.93,Three,"""If you have a heart, if you have a soul, Kare...",False,19
8,The Boys in the Boat: Nine Americans and Their...,£22.60,Four,For readers of Laura Hillenbrand's Seabiscuit ...,False,19
9,The Black Maria,£52.15,One,"Praise for Aracelis Girmay:""[Girmay's] every l...",False,19


In [8]:
# Veriyle alakalı teknik bilgileri (hangi veri tiplerinden olduklarını, ne kadar veri olduğunu vb) inceleyelim
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         1000 non-null   object
 1   price        1000 non-null   object
 2   starRating   1000 non-null   object
 3   description  998 non-null    object
 4   stockStatus  1000 non-null   bool  
 5   stockAmount  1000 non-null   int64 
dtypes: bool(1), int64(1), object(4)
memory usage: 40.2+ KB


In [10]:
# Kitap açıklamaları olmayan kitapları modeli yanlış etkilememesi adına veri setinden uzaklaştıracağız
df[df["description"].isna()]

,name,price,starRating,description,stockStatus,stockAmount
160,The Bridge to Consciousness: I'm Writing the B...,£32.00,Three,NaN,False,15
995,Alice in Wonderland (Alice's Adventures in Won...,£55.53,One,NaN,False,1


In [12]:
df.dropna(inplace=True)

In [36]:
df.reset_index(drop=True, inplace=True)

In [13]:
# Artık verisetinde açıklaması olmayan kitap kalmadı
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 998 entries, 0 to 999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         998 non-null    object
 1   price        998 non-null    object
 2   starRating   998 non-null    object
 3   description  998 non-null    object
 4   stockStatus  998 non-null    bool  
 5   stockAmount  998 non-null    int64 
dtypes: bool(1), int64(1), object(4)
memory usage: 47.8+ KB


**Toplam kitap bilgisini oluşturduğumuz DataFrame'in uzunluğundan kolayca bulabiliriz**

In [14]:
print(f"Toplam {len(df)} kitap incelenecek.")

Toplam 998 kitap incelenecek.


### **Data Preprocessing ( Veri Önişleme )**

In [15]:
def preprocess_text(text):
  if not isinstance(text, str):
    return "" # Eğer veriyi önişleme aşamasında açıklaması boş veya nan kitap kaldıysa önişlemeye almıyoruz
  text = text.lower()
  text = re.sub(r"[^a-z0-9 ]", "", text)  # Özel karakterleri temizleme
  text = re.sub(r"\s+", " ", text).strip()  # Gereksiz boşlukları kaldırma
  return text

In [17]:
# Tüm açıklamalar için metin önişleme methodunu lambda function sayesinde uygulayalım
df["description"] = df["description"].apply(preprocess_text)

In [18]:
df["description"].head(10)

,description
0,its hard to imagine a world without a light in...
1,erotic and absorbingwritten with starling powe...
2,dans une france assez proche de la ntre un hom...
3,wicked above her hipbone girl across her heart...
4,from a renowned historian comes a groundbreaki...
5,patient twentyninea monster roams the halls of...
6,drawing on his extensive experience evaluating...
7,if you have a heart if you have a soul karen h...
8,for readers of laura hillenbrands seabiscuit a...
9,praise for aracelis girmaygirmays every losssh...


### **Embedding Vector (Metinleri Numerik Feature Vektörlere Transforme etme )**

In [27]:
descriptions = [description for description in df["description"]]
tfidf_vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(descriptions)

tfidf_similarities = cosine_similarity(tfidf_matrix)
tfidf_similarities

array([[1.        , 0.        , 0.        , ..., 0.00361885, 0.00237592,
        0.01925318],
       [0.        , 1.        , 0.        , ..., 0.04945523, 0.0340118 ,
        0.05098485],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.00361885, 0.04945523, 0.        , ..., 1.        , 0.02155376,
        0.00528143],
       [0.00237592, 0.0340118 , 0.        , ..., 0.02155376, 1.        ,
        0.0302313 ],
       [0.01925318, 0.05098485, 0.        , ..., 0.00528143, 0.0302313 ,
        1.        ]])

In [46]:
count_vectorizer = CountVectorizer(stop_words="english", max_features=5000)
count_matrix = count_vectorizer.fit_transform(descriptions)

bow_similarities = cosine_similarity(count_matrix)

In [47]:
bow_similarities

array([[1.        , 0.        , 0.        , ..., 0.01590611, 0.00990148,
        0.06499254],
       [0.        , 1.        , 0.        , ..., 0.03794855, 0.06141923,
        0.06891473],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.01590611, 0.03794855, 0.        , ..., 1.        , 0.03212877,
        0.01757424],
       [0.00990148, 0.06141923, 0.        , ..., 0.03212877, 1.        ,
        0.0437595 ],
       [0.06499254, 0.06891473, 0.        , ..., 0.01757424, 0.0437595 ,
        1.        ]])

In [55]:
bert_model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = bert_model.encode(descriptions, convert_to_tensor=True)
# df["bert_embedding"] = list(bert_embeddings.cpu().numpy())
# df["bert_embedding"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

,bert_embedding
0,"[0.0046960646, -0.050254695, 0.05881876, 0.021..."
1,"[0.025182758, -0.059178837, 0.056617755, 0.005..."
2,"[0.0011000833, 0.042354558, -0.04997307, -0.07..."
3,"[0.0036815328, -0.06675138, -0.032670945, 0.01..."
4,"[-0.08089764, 0.031362284, 0.007286476, -0.003..."
...,...
993,"[0.023947056, 0.041257907, -0.10877375, 0.0567..."
994,"[-0.010163322, 0.069646135, -0.012118677, -0.0..."
995,"[-0.049990706, -0.094549574, -0.0025664556, -0..."
996,"[-0.05068499, -0.16166177, -0.054585233, -0.00..."


In [57]:
tfidf_similarities = cosine_similarity(tfidf_matrix)
bow_similarities = cosine_similarity(count_matrix)
bert_similarities = cosine_similarity(bert_embeddings.cpu().numpy())

# 8. EN BENZER 5 KİTABI BULMA
def get_top_similar_books(similarity_matrix, df, vectorizer_type, top_n=5):
    similar_books = {}
    for idx in range(len(df)):
        similar_indices = np.argsort(similarity_matrix[idx])[::-1][1:top_n+1]
        similar_books[df.loc[idx, "name"]] = df.loc[similar_indices, "name"].tolist()
        df[f"top{top_n} similar books {vectorizer_type}"] = df["name"].map(similar_books)
    return similar_books

tfidf_top5 = get_top_similar_books(tfidf_similarities, df, "TF-IDF")
bow_top5 = get_top_similar_books(bow_similarities, df, "CountVectorizer")
bert_top5 = get_top_similar_books(bert_similarities, df, "BERT")

In [60]:
df.head()

,name,price,starRating,description,stockStatus,stockAmount,top5 similar books TF-IDF,top5 similar books CountVectorizer,top5 similar books BERT
0,A Light in the Attic,£51.77,Three,its hard to imagine a world without a light in...,False,22,"[Twenty Love Poems and a Song of Despair, You ...","[Twenty Love Poems and a Song of Despair, Sens...","[Untitled Collection: Sabbath Poems 2014, Hous..."
1,Tipping the Velvet,£53.74,One,erotic and absorbingwritten with starling powe...,False,20,"[Large Print Heart of the Pride, Once Was a Ti...","[When Breath Becomes Air, Fool Me Once, Metern...","[Twenties Girl, Lola and the Boy Next Door (An..."
2,Soumission,£50.10,One,dans une france assez proche de la ntre un hom...,False,20,"[32 Yolks, A la Mode: 120 Recipes in 60 Pairin...","[A la Mode: 120 Recipes in 60 Pairings: Pies, ...",[The Age of Genius: The Seventeenth Century an...
3,Sharp Objects,£47.82,Four,wicked above her hipbone girl across her heart...,False,20,"[The Marriage of Opposites, Girl, Interrupted,...","[Girl, Interrupted, Three Wishes (River of Tim...","[The Girl In The Ice (DCI Erika Foster #1), A ..."
4,Sapiens: A Brief History of Humankind,£54.23,Five,from a renowned historian comes a groundbreaki...,False,20,[Are We Smart Enough to Know How Smart Animals...,"[House of Lost Worlds: Dinosaurs, Dynasties, a...",[Unbound: How Eight Technologies Made Us Human...


In [61]:
book_similarity_results = []
for _, row in df.iterrows():
  book_similarity_results.append(
      {
          "book_name": row["name"],
          "tfidf_top5_similar": row["top5 similar books TF-IDF"],
          "bow_top5_similar": row["top5 similar books CountVectorizer"],
          "bert_top5_similar": row["top5 similar books BERT"]
      }
  )

In [62]:
book_similarity_results

[{'book_name': 'A Light in the Attic',
  'tfidf_top5_similar': ['Twenty Love Poems and a Song of Despair',
   "You can't bury them all: Poems",
   'See America: A Celebration of Our National Parks & Treasured Sites',
   'Whole Lotta Creativity Going On: 60 Fun and Unusual Exercises to Awaken and Strengthen Your Creativity',
   'salt.'],
  'bow_top5_similar': ['Twenty Love Poems and a Song of Despair',
   'Sense and Sensibility',
   'The Stand',
   'The Course of Love',
   'Crazy Love: Overwhelmed by a Relentless God'],
  'bert_top5_similar': ['Untitled Collection: Sabbath Poems 2014',
   'House of Leaves',
   'Shrunken Treasures: Literary Classics, Short, Sweet, and Silly',
   "You can't bury them all: Poems",
   'Poems That Make Grown Women Cry']},
 {'book_name': 'Tipping the Velvet',
  'tfidf_top5_similar': ['Large Print Heart of the Pride',
   'Once Was a Time',
   'Musicophilia: Tales of Music and the Brain',
   'This Is Your Brain on Music: The Science of a Human Obsession',
   'H

In [63]:
# Benzerlikleri gruplandırılmış kitapları JSON dosyası olarak kaydetme
output_file_name = "book_similarity_results.json"
with open(PATH.replace("data.json", output_file_name), "w", encoding="utf-8") as output_file:
  json.dump(book_similarity_results, output_file, indent=4)